In [1]:
from collections import deque, defaultdict, Counter
from heapq import heapify, heappush, heappop
import numpy as np
from copy import deepcopy
import math
import time
from functools import cache, reduce, cmp_to_key
import graphviz
from itertools import product
import matplotlib.pyplot as plt
from bisect import bisect_left, bisect_right
import json
import os
import re
from typing import Any
from dataclasses import dataclass

In [2]:
def print_grid(grid: list[list[str]]):
    print("\n".join("".join(line) for line in grid))


def plot_grid(
    grid: list[list[str]],
    colors: dict[str, int],
    save: bool = False,
    filepath: str = "images/plot.png",
) -> None:
    arr = np.zeros((len(grid), len(grid[0])))
    for r, row in enumerate(grid):
        for c, char in enumerate(row):
            if char in colors:
                arr[r, c] = colors[char]
    plt.xticks([])
    plt.yticks([])
    if save:
        plt.imsave(filepath, arr)
    else:
        plt.imshow(arr)


def plot_objects(
    object_lists: list[list[tuple[int, int]]],
    colors: list[int],
    x_limit: int,
    y_limit: int,
    save: bool = False,
    filepath: str = "images/plot.png",
) -> None:
    arr = np.zeros((y_limit, x_limit))
    for objects, color in zip(object_lists, colors):
        for obj in objects:
            arr[y_limit - 1 - obj[1], obj[0]] = color
    plt.xticks([])
    plt.yticks([])
    if save:
        plt.imsave(filepath, arr)
    else:
        plt.imshow(arr)

In [3]:
dirs4 = [(-1, 0), (0, 1), (1, 0), (0, -1)]
dirs8 = [(-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1)]

today = os.path.basename(globals()["__vsc_ipynb_file__"]).split(".")[0]  # + "_ex"
today

'day18'

In [4]:
def get_lines() -> list[str]:
    lines = []
    with open(f"./data/{today}.txt") as f:
        while line := f.readline():
            lines.append(line.rstrip())
    return lines


def get_grid() -> list[list[str]]:
    grid = []
    with open(f"./data/{today}.txt") as f:
        while line := f.readline():
            grid.append([c for c in line.rstrip()])
    return grid


def parse_nums(s: str) -> list[int]:
    return [int(x) for x in re.findall(r"-?\d+", s)]


def get_nums() -> list[list[int]]:
    lines = get_lines()
    return [parse_nums(line) for line in lines]


def is_inside_grid(coords: tuple[int, int], grid: list[list[Any]]) -> bool:
    return coords[0] in range(len(grid)) and coords[1] in range(len(grid[0]))

In [5]:
x_limit = 71
y_limit = 71
grid_init = [["." for _ in range(y_limit)] for _ in range(x_limit)]

In [6]:
obstacles = get_nums()
len(obstacles), obstacles[:2]

(3450, [[67, 61], [15, 16]])

In [7]:
grid = deepcopy(grid_init)
for x, y in obstacles[:1024]:
    grid[y][x] = "#"

In [8]:
dq = deque([(0, 0, 0)])
seen = set()
while dq:
    r, c, d = dq.popleft()
    if r == y_limit - 1 and c == y_limit - 1:
        print(d)
        break
    if (r, c) in seen:
        continue
    seen.add((r, c))

    for dr, dc in dirs4:
        if not is_inside_grid((r+dr, c+dc), grid):
            continue
        if grid[r+dr][c+dc] == "#":
            continue
        dq.append((r+dr, c+dc, d+1))

284


In [9]:
def is_connected(grid: list[list[str]]) -> bool:
    dq = deque([(0, 0)])
    seen = set()
    while dq:
        r, c = dq.popleft()
        if r == y_limit - 1 and c == y_limit - 1:
            return True
        if (r, c) in seen:
            continue
        seen.add((r, c))

        for dr, dc in dirs4:
            if not is_inside_grid((r+dr, c+dc), grid):
                continue
            if grid[r+dr][c+dc] == "#":
                continue
            dq.append((r+dr, c+dc))
    
    return False

In [10]:
for i in range(1024, len(obstacles)):
    x, y = obstacles[i]
    grid[y][x] = "#"
    if not is_connected(grid):
        print(obstacles[i])
        break

[51, 50]


# Part 2 - Union-Find

In [11]:
free_cells = set([(x, y) for x in range(x_limit) for y in range(y_limit)])

blocked = defaultdict(int)
for x, y in obstacles:
    blocked[(x, y)] += 1
    free_cells.remove((x, y))

In [12]:
parent = {}
for cell in free_cells:
    parent[cell] = cell

def union(pos1: tuple[(int, int)], pos2: tuple[(int, int)]) -> None:
    p1 = find(pos1)
    p2 = find(pos2)
    if p1 == p2:
        return
    parent[pos1] = p2
    parent[p1] = p2

def find(pos: tuple[(int, int)]) -> tuple[(int, int)]:
    if parent[pos] == pos:
        return pos
    p = find(parent[pos])
    parent[pos] = p
    return p

for (x, y) in free_cells:
    for dx, dy in dirs4:
        if (x+dx, y+dy) in free_cells:
            union((x, y), (x+dx, y+dy))

In [13]:
for i in range(len(obstacles) - 1, -1, -1):
    x, y = obstacles[i][0], obstacles[i][1]
    blocked[(x, y)] -= 1
    if blocked[(x, y)] == 0:
        free_cells.add((x, y))
        parent[(x, y)] = (x, y)
        for dx, dy in dirs4:
            if (x+dx, y+dy) in free_cells:
                union((x, y), (x+dx, y+dy))
        if find((0, 0)) == find((x_limit - 1, y_limit - 1)):
            print(f"{x},{y}")
            flag = True
            break

51,50
